In [ ]:
#pip install openai
#export OPENAI_API_KEY="your_key"

In [ ]:
import argparse
import json
import os
import random
import time
from datetime import datetime, timezone

from openai import OpenAI

In [ ]:
BASE_PROMPT = """

"""

In [ ]:
REALITY_FRAMES = {
    "everyday": (
        "Describe the situation as a realistic, common-sense workplace moral dilemma. "
        "Focus on emotions, social dynamics, and practical consequences."
    ),
    "scientific": (
        "Explain the same situation from a scientific/empirical viewpoint. "
        "Use detached, objective reasoning (e.g., incentives, organizational behavior, "
        "risk analysis, decision theory). Avoid melodrama."
    ),
    "imaginative": (
        "Reimagine the dilemma in a fantasy world with fictional institutions and stakes. "
        "Keep the core moral conflict analogous to the original."
    ),
}

In [ ]:
def build_prompt(frame_name: str) -> str:
    return (
        f"Reality frame: {frame_name}\n"
        f"Scenario: {BASE_PROMPT}\n\n"
        f"Instruction: {REALITY_FRAMES[frame_name]}\n\n"
        "Write a single short narrative of about 300–400 words.\n"
        "Do not add a title. Do not use bullet points.\n"
    )


In [ ]:
def generate_one(client: OpenAI, model: str, prompt: str, temperature: float) -> str:
    # Responses API (recommended)
    resp = client.responses.create(
        model=model,
        input=prompt,
        temperature=temperature,
    )
    return resp.output_text

In [ ]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", type=str, default="gpt-4.1-mini", help="Model name")
    parser.add_argument("--repeats", type=int, default=5, help="Repeats per frame (5–10 recommended)")
    parser.add_argument("--out", type=str, default="outputs.jsonl", help="Output JSONL path")
    parser.add_argument("--min_temp", type=float, default=0.7, help="Min temperature")
    parser.add_argument("--max_temp", type=float, default=1.0, help="Max temperature")
    parser.add_argument("--sleep", type=float, default=0.2, help="Sleep seconds between calls")
    args = parser.parse_args()

    if not os.getenv("OPENAI_API_KEY"):
        raise SystemExit("Missing OPENAI_API_KEY env var. Set it and rerun.")

    client = OpenAI()

    random.seed(42)
    total = 0

    with open(args.out, "w", encoding="utf-8") as f:
        for frame in REALITY_FRAMES.keys():
            for i in range(args.repeats):
                temp = random.uniform(args.min_temp, args.max_temp)
                prompt = build_prompt(frame)

                try:
                    text = generate_one(client, args.model, prompt, temp)
                except Exception as e:
                    # Basic resilience: log the error and continue
                    record = {
                        "timestamp": datetime.now(timezone.utc).isoformat(),
                        "frame": frame,
                        "repeat_index": i,
                        "model": args.model,
                        "temperature": temp,
                        "prompt": prompt,
                        "error": str(e),
                    }
                    f.write(json.dumps(record, ensure_ascii=False) + "\n")
                    f.flush()
                    time.sleep(max(args.sleep, 1.0))
                    continue

                record = {
                    "timestamp": datetime.now(timezone.utc).isoformat(),
                    "frame": frame,
                    "repeat_index": i,
                    "model": args.model,
                    "temperature": temp,
                    "prompt": prompt,
                    "output": text,
                    "word_count": len(text.split()),
                }
                f.write(json.dumps(record, ensure_ascii=False) + "\n")
                f.flush()

                total += 1
                time.sleep(args.sleep)

    print(f"Done. Wrote {total} generations to {args.out}")
